In [114]:
import pandas as pd
import numpy as np
import math

In [115]:
employees_df = pd.read_json('employees.json')
employees_df

,employee_id,name,age,salary,hire_date
0,101,Alice Smith,25.0,50000.0,2020-01-15
1,102,Bob Johnson,30.0,60000.0,2019-06-20
2,103,Charlie Brown,NaN,75000.0,2021-03-10
3,104,None,40.0,NaN,2018-11-01
4,105,Eve Davis,35.0,55000.0,2020-09-25


In [116]:
departments_df = pd.read_json('departments.json')
departments_df

,employee_id,department
0,101,HR
1,102,IT
2,103,Finance
3,106,Marketing


## 1) Fill missing name values with 'Unknown'.

In [117]:
# Check wheather we have null values in all our columns of employees table
employees_df.isnull().sum()

employee_id    0
name           1
age            1
salary         1
hire_date      0
dtype: int64

In [118]:
employees_df['name'] = employees_df['name'].fillna('Unknown')
employees_df

,employee_id,name,age,salary,hire_date
0,101,Alice Smith,25.0,50000.0,2020-01-15
1,102,Bob Johnson,30.0,60000.0,2019-06-20
2,103,Charlie Brown,NaN,75000.0,2021-03-10
3,104,Unknown,40.0,NaN,2018-11-01
4,105,Eve Davis,35.0,55000.0,2020-09-25


## 2) Fill missing age values with the median age.

In [119]:
employees_df['age'] = employees_df['age'].fillna(np.ceil(employees_df['age'].median()))
employees_df

,employee_id,name,age,salary,hire_date
0,101,Alice Smith,25.0,50000.0,2020-01-15
1,102,Bob Johnson,30.0,60000.0,2019-06-20
2,103,Charlie Brown,33.0,75000.0,2021-03-10
3,104,Unknown,40.0,NaN,2018-11-01
4,105,Eve Davis,35.0,55000.0,2020-09-25


## 3) Fill missing salary values with the mean salary.

In [120]:
employees_df['salary'] = employees_df['salary'].fillna(employees_df['salary'].mean())
employees_df

,employee_id,name,age,salary,hire_date
0,101,Alice Smith,25.0,50000.0,2020-01-15
1,102,Bob Johnson,30.0,60000.0,2019-06-20
2,103,Charlie Brown,33.0,75000.0,2021-03-10
3,104,Unknown,40.0,60000.0,2018-11-01
4,105,Eve Davis,35.0,55000.0,2020-09-25


In [121]:
employees_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   employee_id  5 non-null      int64  
 1   name         5 non-null      object 
 2   age          5 non-null      float64
 3   salary       5 non-null      float64
 4   hire_date    5 non-null      object 
dtypes: float64(2), int64(1), object(2)
memory usage: 332.0+ bytes


## 4) Add a years_employed column, calculated as the number of years from hire_date to the current date (October 9, 2025).

In [122]:
employees_df['hire_date'] = pd.to_datetime(employees_df['hire_date'])

In [124]:
current_date = pd.to_datetime('2025-10-09')

In [133]:
employees_df['years_employed'] = ((current_date - employees_df['hire_date']).dt.days/365.25).round(1)
employees_df

,employee_id,name,age,salary,hire_date,years_employed
0,101,Alice Smith,25.0,50000.0,2020-01-15,5.7
1,102,Bob Johnson,30.0,60000.0,2019-06-20,6.3
2,103,Charlie Brown,33.0,75000.0,2021-03-10,4.6
3,104,Unknown,40.0,60000.0,2018-11-01,6.9
4,105,Eve Davis,35.0,55000.0,2020-09-25,5.0


## 5) Add a salary_category column where:
Salary < 55000: 'Low'
Salary >= 55000 and < 70000: 'Medium'
Salary >= 70000: 'High'

In [139]:
def salary_cat(salary):
    if salary < 55000 :
        return 'Low'
    elif salary >= 55000 and salary < 70000:
        return 'Medium'
    else:
        return 'High'


employees_df['salary_category'] = [salary_cat(s) for s in employees_df['salary']]
employees_df

,employee_id,name,age,salary,hire_date,years_employed,salary_category
0,101,Alice Smith,25.0,50000.0,2020-01-15,5.7,Low
1,102,Bob Johnson,30.0,60000.0,2019-06-20,6.3,Medium
2,103,Charlie Brown,33.0,75000.0,2021-03-10,4.6,High
3,104,Unknown,40.0,60000.0,2018-11-01,6.9,Medium
4,105,Eve Davis,35.0,55000.0,2020-09-25,5.0,Medium


## 6) Merge with department data using an inner join on employee_id.

In [140]:
merged_df = pd.merge(employees_df, departments_df, on = 'employee_id', how = 'inner')
merged_df

,employee_id,name,age,salary,hire_date,years_employed,salary_category,department
0,101,Alice Smith,25.0,50000.0,2020-01-15,5.7,Low,HR
1,102,Bob Johnson,30.0,60000.0,2019-06-20,6.3,Medium,IT
2,103,Charlie Brown,33.0,75000.0,2021-03-10,4.6,High,Finance


## 7) Filter to include only employees with age >= 30.

In [142]:
filtered_df = merged_df[merged_df['age'] >= 30]
filtered_df

,employee_id,name,age,salary,hire_date,years_employed,salary_category,department
1,102,Bob Johnson,30.0,60000.0,2019-06-20,6.3,Medium,IT
2,103,Charlie Brown,33.0,75000.0,2021-03-10,4.6,High,Finance


## 8) Sort by salary in descending order.

In [146]:
sorted_df = filtered_df.sort_values(by='salary', ascending=False)
sorted_df

,employee_id,name,age,salary,hire_date,years_employed,salary_category,department
2,103,Charlie Brown,33.0,75000.0,2021-03-10,4.6,High,Finance
1,102,Bob Johnson,30.0,60000.0,2019-06-20,6.3,Medium,IT


## 9) Save the result to transformed_employees.csv with Pipe Delimited and Header.

In [147]:
sorted_df.to_csv('transformed_employees.csv', sep='|', header=True)